In [1]:
import matplotlib.pyplot as plt
import healpy as hp
import numpy as np
import sys
import os
sys.path.append('..')

In [2]:
%load_ext autoreload
%autoreload 2
from lat_cb.signal import *
from lat_cb.spectra import *
from lat_cb.mle import *

In [3]:
libdir = '/Users/antoidicherianlonappan/Workspace/scratch'
nside = 512
alpha = 0.35
dust = 1
synch = 1
beta = [0.1,0.1,0.8,0.8,1.2,1.2]
lmax = 1000
lat = Spectra(libdir,nside,alpha,dust,synch,beta,lmax)

In [4]:
lat.compute(0)

sync x obs spectra: 100%|██████████| 6/6 [00:00<00:00, 1405.21band/s]


In [5]:
ml = MLE(libdir,nside,alpha,dust,synch,beta,lmax)

In [8]:
ml.estimate_angle(0)

{'alpha_27': 0.16665531215282284,
 'alpha_39': 0.1442174201428122,
 'alpha_93': 0.8411039776305793,
 'alpha_145': 0.863463844171913,
 'alpha_225': 1.2724231800475165,
 'alpha_280': 1.2738358112453119,
 'beta': 0.3534345909987296}